# Lab Assignment 10: Exploratory Data Analysis, Part 1
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will be working with the 2018 [General Social Survey (GSS)](http://www.gss.norc.org/). The GSS is a sociological survey created and regularly collected since 1972 by the National Opinion Research Center at the University of Chicago. It is funded by the National Science Foundation. The GSS collects information and keeps a historical record of the concerns, experiences, attitudes, and practices of residents of the United States, and it is one of the most important data sources for the social sciences. 

The data includes features that measure concepts that are notoriously difficult to ask about directly, such as religion, racism, and sexism. The data also include many different metrics of how successful a person is in his or her profession, including income, socioeconomic status, and occupational prestige. These occupational prestige scores are coded separately by the GSS.  The full description of their methodology for measuring prestige is available here: http://gss.norc.org/Documents/reports/methodological-reports/MR122%20Occupational%20Prestige.pdf Here's a quote to give you an idea about how these scores are calculated:

> Respondents then were given small cards which each had a single occupational titles listed on it. Cards were in English or Spanish. They were given one card at a time in the preordained order. The interviewer then asked the respondent to "please put the card in the box at the top of the ladder if you think that occupation has the highest possible social standing. Put it in the box of the bottom of the ladder if you think it has the lowest possible social standing. If it belongs somewhere in between, just put it in the box that matches the social standing of the occupation."

The prestige scores are calculated from the aggregated rankings according to the method described above.

### Problem 0
Import the following packages:

In [3]:
import numpy as np
import pandas as pd
import sidetable
import weighted # this is a module of wquantiles, so type pip install wquantiles or conda install wquantiles to get access to it
from scipy import stats 
from sklearn import manifold
from sklearn import metrics
import prince
from pandas_profiling import ProfileReport
pd.options.display.max_columns = None

Then load the GSS data with the following code:

In [4]:
%%capture
gss = pd.read_csv("https://github.com/jkropko/DS-6001/raw/master/localdata/gss2018.csv",
                 encoding='cp1252', na_values=['IAP','IAP,DK,NA,uncodeable', 'NOT SURE',
                                               'DK', 'IAP, DK, NA, uncodeable', '.a', "CAN'T CHOOSE"])

### Problem 1
Drop all columns except for the following:
* `id` - a numeric unique ID for each person who responded to the survey
* `wtss` - survey sample weights
* `sex` - male or female
* `educ` - years of formal education
* `region` - region of the country where the respondent lives
* `age` - age
* `coninc` - the respondent's personal annual income
* `prestg10` - the respondent's occupational prestige score, as measured by the GSS using the methodology described above
* `mapres10` - the respondent's mother's occupational prestige score, as measured by the GSS using the methodology described above
* `papres10` -the respondent's father's occupational prestige score, as measured by the GSS using the methodology described above
* `sei10` - an index measuring the respondent's socioeconomic status
* `satjob` - responses to "On the whole, how satisfied are you with the work you do?"
* `fechld` - agree or disagree with: "A working mother can establish just as warm and secure a relationship with her children as a mother who does not work."
* `fefam` - agree or disagree with: "It is much better for everyone involved if the man is the achiever outside the home and the woman takes care of the home and family."
* `fepol` - agree or disagree with: "Most men are better suited emotionally for politics than are most women."
* `fepresch` - agree or disagree with: "A preschool child is likely to suffer if his or her mother works."
* `meovrwrk` - agree or disagree with: "Family life often suffers because men concentrate too much on their work."

Then rename any columns with names that are non-intuitive to you to more intuitive and descriptive ones. Finally, replace the "89 or older" values of `age` with 89, and convert `age` to a float data type. [1 point]

In [6]:
# Create a list to hold selected features names -- total 17 of them:
selected_fetures = ['id', 'wtss', 'sex', 'educ', 
                    'region', 'age', 'coninc', 'prestg10', 
                    'mapres10', 'papres10', 'sei10', 'satjob', 
                    'fechld', 'fefam', 'fepol', 'fepresch', 'meovrwrk']
len(selected_fetures)

17

In [5]:
gss.head()

abany abdefect    abfelegl abhelp1 abhelp2 abhelp3 abhelp4 abhlth  \
0    no      yes         NaN     yes     yes     yes     yes    yes   
1   yes      yes  it depends      no      no      no      no    yes   
2   NaN      NaN         NaN     yes      no     yes     yes    NaN   
3   NaN      NaN      should     yes     yes     yes     yes    NaN   
4    no      yes         NaN      no      no      no     yes    yes   

                    abinspay  \
0      people should be able   
1  people should not be able   
2  people should not be able   
3      people should be able   
4  people should not be able   

                                           abmedgov1  \
0                       the government should decide   
1                                                NaN   
2  a woman and her medical professional should de...   
3                                                NaN   
4                                                NaN   

                                           abmedgov2    abmelegl  \
0                                                NaN  it depends   
1  a woman and her medical professional should de...         NaN   
2                                                NaN  it depends   
3  a woman and her medical professional should de...         NaN   
4  a woman and her medical professional should de...  it depends   

           abmoral abnomore abpoor               abpoorw abrape absingle  \
0  morally opposed       no     no          always wrong    yes       no   
1       it depends      yes     no                   NaN    yes      yes   
2       it depends      NaN    NaN  wrong only sometimes    NaN      NaN   
3       it depends      NaN    NaN                   NaN    NaN      NaN   
4  morally opposed       no     no                   NaN     no       no   

                abstate1              abstate2 acqntsex    actssoc  \
0  neither easy nor hard        make it harder      NaN  very good   
1                   easy  stay the same as now      NaN       good   
2              very easy        make it harder      NaN  excellent   
3                   easy  stay the same as now      NaN  very good   
4             Don't know  stay the same as now      NaN  excellent   

                                  adminconsent adults        advfront  \
0  r does not consent to possible data linkage      5  strongly agree   
1  r does not consent to possible data linkage      2             NaN   
2  r does not consent to possible data linkage      2        disagree   
3          r consents to possible data linkage      2             NaN   
4          r consents to possible data linkage      2             NaN   

               affrmact             afraidof         afterlif age  \
0  strongly oppose pref                never  yes, definitely  43   
1                   NaN                never              NaN  74   
2  strongly oppose pref                  NaN  yes, definitely  42   
3           oppose pref                  NaN              NaN  63   
4                   NaN  a few times a month              NaN  71   

          aged  agekdbrn            ancestrs arthrtis astrolgy  \
0          NaN       NaN  no, definitely not       no       no   
1  a good idea      21.0                 NaN      NaN      NaN   
2  a good idea      35.0    no, probably not       no       no   
3  a good idea      32.0                 NaN      yes      NaN   
4   a bad idea       NaN                 NaN      NaN      NaN   

                astrosci                       atheists            attend  \
0  not at all scientific              somewhat negative      2-3x a month   
1                    NaN                            NaN       once a year   
2  not at all scientific  neither positive nor negative       once a year   
3                    NaN                            NaN   nrly every week   
4                    NaN                            NaN  more thn once wk   

               attend12              attendm

### Problem 2
#### Part a
Use the `ProfileReport()` function to generate and embed an HTML formatted exploratory data analysis report in your notebook. Make sure that it includes a "Correlations" report along with "Overview" and "Variables". [1 point]

#### Part b
Looking through the HTML report you displayed in part a, how many people in the data are from New England? [1 point]

#### Part c
Looking through the HTML report you displayed in part a, which feature in the data has the highest number of missing values, and what percent of the values are missing for this feature? [1 point]

#### Part d
Looking through the HTML report you displayed in part a, which two distinct features in the data have the highest correlation? [1 point]

### Problem 3
On a primetime show on a 24-hour cable news network, two unpleasant-looking men in suits sit across a table from each other, scowling. One says "This economy is failing the middle-class. The average American today is making less than \\$48,000 a year." The other screams "Fake news! The typical American makes more than \$55,000 a year!" Explain, using words and code, how the data can support both of their arguments. Use the sample weights to calculate descriptive statistics that are more representative of the American adult population as a whole. [1 point]

### Problem 4
For each of the following parts, 
* generate a table that provides evidence about the relationship between the two features in the data that are relevant to each question, 
* interpret the table in words, 
* use a hypothesis test to assess the strength of the evidence in the table, 
* and provide a **specific and accurate** intepretation of the $p$-value associated with this hypothesis test beyond "significant or not". 

#### Part a
Is there a gender wage gap? That is, is there a difference between the average incomes of men and women? [2 points]

#### Part b
Are there different average values of occupational prestige for different levels of job satisfaction? [2 points]

### Problem 5
Report the Pearson's correlation between years of education, socioeconomic status, income, occupational prestige, and a person's mother's and father's occupational prestige? Then perform a hypothesis test for the correlation between years of education and socioeconomic status and provide a **specific and accurate** intepretation of the $p$-value associated with this hypothesis test beyond "significant or not". [2 points]

### Problem 6
Create a new categorical feature for age groups, with categories for 18-35, 36-49, 50-69, and 70 and older (see the module 8 notebook for an example of how to do this). 

Then create a cross-tabulation in which the rows represent age groups and the columns represent responses to the statement that "It is much better for everyone involved if the man is the achiever outside the home and the woman takes care of the home and family." Rearrange the columns so that they are in the following order: strongly agree, agree, disagree, strongly disagree. Place row percents in the cells of this table.

Finally, use a hypothesis test that can tell use whether there is enough evidence to conclude that these two features have a relationship, and provide a specific and accurate intepretation of the $p$-value. [2 points]

### Problem 7
For this problem, you will conduct and interpret a correspondence analysis on the categorical features that ask respondents to state the extent to which they agree or disagree with the statements:
* "A working mother can establish just as warm and secure a relationship with her children as a mother who does not work."
* "It is much better for everyone involved if the man is the achiever outside the home and the woman takes care of the home and family."
* "Most men are better suited emotionally for politics than are most women."
* "A preschool child is likely to suffer if his or her mother works."
* "Family life often suffers because men concentrate too much on their work."

#### Part a
Conduct a correspondence analysis using the observed features listed above that measures two latent features. Plot the two latent categories for each category in each of the features used in the analysis. [2 points]

#### Part b
Display the latent features for every category in the observed features, sorted by the first latent feature. Describe in words what concept this feature is attempting to measure, and give the feature a name. [2 points]

#### Part c
We can use the results of the MCA model to conduct some cool EDA. For one example, follow these steps:

1. Use the `.row_coordinates()` method to calculate values of the latent feature for every row in the data you passed to the MCA in part a. Extract the first column and store it in its own dataframe.

2. To join it with the full, cleaned GSS data based on row numbers (instead of on a primary key), use the `.join()` method. For example, if we named the cleaned GSS data `gss_clean` and if we named the dataframe in step 1 `latentfeature`, we can type
```
gss_clean = gss_clean.join(latentfeature, how="outer")
```
3. Create a cross-tabuation with age categories (that you constructed in problem 5) in the rows and sex in the columns. Instead of a frequency, place the mean value of the latent feature in the cells. 

What does this table tell you about the relationship between sex, age, and the latent feature? [2 points]